1- Detect and crop license plates in car images.

https://www.kaggle.com/datasets/andrewmvd/car-plate-detection?select=annotations

2- detect and crop characters from license plates images.

3- classify characters and print text output.

In [1]:
# Imports

!pip install tensorflow==2.4.0
!pip install tensorflow-gpu==2.4.0
!pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0
!pip install imageai --upgrade
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import PIL.Image
from PIL import ImageOps
import PIL
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import subprocess
import sys
from imageai.Classification import ImageClassification
from imageai.Detection.Custom import DetectionModelTrainer
from imageai.Detection.Custom import CustomObjectDetection
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="/content/drive/MyDrive/imageai/LPDS")
trainer.setTrainConfig(object_names_array=["licence"], batch_size=5, num_experiments=20, train_from_pretrained_model="/content/drive/MyDrive/imageai/resnet50_coco_best_v2.1.0.h5")
trainer.trainModel()

# trainer = DetectionModelTrainer()
# trainer.setModelTypeAsYOLOv3()
# trainer.setDataDirectory(data_directory="/content/drive/MyDrive/imageai/LPDS")
# metrics = trainer.evaluateModel(model_path="/content/drive/MyDrive/imageai/LPDS/models", json_path="/content/drive/MyDrive/imageai/LPDS/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)
# print(metrics)

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.72
Anchor Boxes generated.
Detection configuration saved in  /content/drive/MyDrive/imageai/LPDS/json/detection_config.json
Evaluating over 101 samples taken from /content/drive/MyDrive/imageai/LPDS/validation
Training over 332 samples  given at /content/drive/MyDrive/imageai/LPDS/train
Training on: 	['licence']
Training with Batch Size:  5
Number of Training Samples:  332
Number of Validation Samples:  101
Number of Experiments:  20
Training with transfer learning from pretrained Model


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/20
536/536 [==============================] - 443s 808ms/step - loss: 59.6049 - yolo_layer_loss: 7.3327 - yolo_layer_1_loss: 17.1698 - yolo_layer_2_loss: 23.7922 - val_loss: 31.7920 - val_yolo_layer_loss: 3.7460 - val_yolo_layer_1_loss: 8.5456 - val_yolo_layer_2_loss: 11.0768
Epoch 2/20
536/536 [==============================] - 353s 657ms/step - loss: 23.5950 - yolo_layer_loss: 2.9304 - yolo_layer_1_loss: 5.9551 - yolo_layer_2_loss: 7.1046 - val_loss: 32.2135 - val_yolo_layer_loss: 6.5893 - val_yolo_layer_1_loss: 9.6832 - val_yolo_layer_2_loss: 9.8265
Epoch 3/20
536/536 [==============================] - 344s 641ms/step - loss: 19.7554 - yolo_layer_loss: 2.8889 - yolo_layer_1_loss: 5.0501 - yolo_layer_2_loss: 5.8850 - val_loss: 17.4429 - val_yolo_layer_loss: 2.0728 - val_yolo_layer_1_loss: 4.5546 - val_yolo_layer_2_loss: 5.2518
Epoch 4/20
536/536 [==============================] - 359s 669ms/step - loss: 19.1068 - yolo_layer_loss: 3.5850 - yolo_layer_1_loss: 4.5150 - yolo_laye

In [6]:
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("/content/drive/MyDrive/imageai/LPDS/models/detection_model-ex-034--loss-0013.373.h5")
detector.setJsonPath("/content/drive/MyDrive/imageai/LPDS/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image="/content/drive/MyDrive/imageai/LPDS/train/images/Cars103.png", output_image_path="/content/drive/MyDrive/imageai/LP-detected.jpg")
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


In [ ]:
prediction = ImageClassification()

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = True, 
    weights = "imagenet"                  
)

saving_path = pathlib.Path('/content/drive/MyDrive/imageai/SavedBaseModel.h5')

base_model.save(saving_path)

model_path = pathlib.Path('/content/drive/MyDrive/imageai/SavedBaseModel.h5')

prediction.setModelTypeAsInceptionV3()

prediction.setModelPath("/content/drive/MyDrive/imageai/SavedBaseModel.h5")

prediction.loadModel()

predictions, probabilities = prediction.classifyImage("/content/drive/MyDrive/License_Plate_Recognition/LicPlateImages/10.png", result_count=10)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)
